In [1]:
# Library imports
import random       # Used for random.choice in MCTS
import copy         # Used for deepcopy in MCTS
import time         # Used for optimization (LIKELY CAN REMOVE LATER)
import numpy as np  # Used for stuff and things
import multiprocessing as mp
import tensorflow as tf # Used for neural networks
import pickle

2023-07-14 12:38:07.436013: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-14 12:38:07.566257: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Class imports
from TTTBoard import TTTBoard
from MCTS import MCTS
#from trainingExample import trainingExample
#from TrainingManager import TrainingManager
#from NetworkArchitectureTester import NetworkArchitectureTester


# Function imports
#from simulateSelfPlayGames import simulateSelfPlayGames


# Config import
from Config import *

#tf.compat.v1.disable_eager_execution()

In [6]:
# manager = TrainingManager()

# file = open("TrainingExamples/Episode_14", 'rb')
# manager.trainingExamples = pickle.load(file)
# file.close()

# manager.train()

In [3]:
def initNNet():
    # Define network architecture
    tic_tac_toe_input = tf.keras.layers.Input(shape=(3,3,3), name='TTT_input')
    flatten = tf.keras.layers.Flatten()(tic_tac_toe_input)
    hidden = tf.keras.layers.Dense(64, activation='relu')(flatten)
    hidden = tf.keras.layers.Dense(64, activation='relu')(hidden)
    policy_output = tf.keras.layers.Dense(9, activation='softmax', name='policy_output')(hidden)
    value_output = tf.keras.layers.Dense(1, activation='tanh', name='value_output')(hidden)
    model = tf.keras.models.Model(inputs=tic_tac_toe_input, outputs=[policy_output, value_output])
    
    #model.summary()
    
    # Compile model 
    losses = {
        'policy_output': 'categorical_crossentropy', 
        'value_output': 'mse'
    }
    
    model.compile(optimizer='Adam', loss=losses, metrics=['accuracy'])
    return model

In [6]:
search = MCTS(NNet=initNNet())

while not search.board.is_game_over():
    search.search()
    print(search.n)
    print(search.pi)
    print(search.board.find_moves())
    child_choice = np.argmax(search.pi)
    #child_choice = np.random.choice(len(search.pi), p=search.pi)
    print(child_choice)
    move_choice = search.children[child_choice].move
    print(move_choice)
    search = search.make_move(move_choice)
    search.board.print()

100
[0.08080808080808081, 0.16161616161616163, 0.050505050505050504, 0.08080808080808081, 0.09090909090909091, 0.12121212121212122, 0.08080808080808081, 0.12121212121212122, 0.21212121212121213]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
8
8
 x  _  _ 

 _  _  _     O's move

 _  _  _ 



121
[0.11666666666666667, 0.1, 0.14166666666666666, 0.13333333333333333, 0.08333333333333333, 0.19166666666666668, 0.15833333333333333, 0.075]
[0, 1, 2, 3, 4, 5, 6, 7]
5
5
 x  _  _ 

 o  _  _     X's move

 _  _  _ 



123
[0.08196721311475409, 0.1557377049180328, 0.04918032786885246, 0.06557377049180328, 0.5245901639344263, 0.06557377049180328, 0.05737704918032787]
[0, 1, 2, 3, 4, 6, 7]
4
4
 x  _  _ 

 o  x  _     O's move

 _  _  _ 



164
[0.43558282208588955, 0.12883435582822086, 0.08588957055214724, 0.09815950920245399, 0.1411042944785276, 0.11042944785276074]
[0, 1, 2, 3, 6, 7]
0
0
 x  _  _ 

 o  x  _     X's move

 _  _  o 



171
[0.047058823529411764, 0.023529411764705882, 0.03529411764705882, 0.041176470588